In [1]:
import torch
import torch.nn as nn
from torch.nn.utils import weight_norm
from torchsummary import summary
from TCN import TemporalConvNet
import argparse
from torch import optim
import torch.nn.functional as F
from torch.utils.data import Dataset ,DataLoader
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook
import wandb
import matplotlib.pyplot as plt
import pickle
import datasets

/home/ssu36/anaconda3/envs/tiger/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [20]:
#data
DEVICE = 'cuda:0'
IR = 0.1
EPOCH =400
DROPOUT=0.3
PATIENCE =20
BATCH_SIZE = 1024
NUM_LAYERS =2
SUFFLE = True
WINDOW_SIZE  = 90
PREDICT_SIZE = 15
SLIDING_SIZE =1
FEATURE_SIZE = 5
BATCH_SIZE =64
FULLLY_SIZE =128
FEATURE = [ 'Open', 'High','Low','Close', 'Volume']
TARGET = 'Close'
TICKER_NUMBER =2136
TOTAL_DAY = 755
WINDOW_NUMBER =int(TOTAL_DAY - (WINDOW_SIZE+PREDICT_SIZE)/SLIDING_SIZE+1)



#model
input_size = 5
num_channels =[80]*4
kenel_size = 7
output_size= 30
dropout = 0.25

In [21]:
class TCNModel(nn.Module):
    def __init__(self, input_size, output_size, num_channels, kernel_size, dropout):
        super(TCNModel, self).__init__()
        self.tcn = TemporalConvNet(input_size, num_channels, kernel_size=kernel_size, dropout=dropout)
        self.linear = nn.Linear(num_channels[-1], output_size)
        self.init_weights()

    def init_weights(self):
        self.linear.weight.data.normal_(0, 0.01)

    def forward(self, x):
        y1 = self.tcn(x)
        return self.linear(y1[:, :, -1])

In [9]:

import polars as pl


ModuleNotFoundError: No module named 'polars'

In [2]:
2//650

0

In [10]:
!pip install polars

  Obtaining dependency information for polars from https://files.pythonhosted.org/packages/0f/3c/6a9783afff586e5ea1c6ea12f462c1d839c1319447fe1e4d9a13e15c885c/polars-0.19.15-cp38-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.4/28.4 MB 3.1 MB/s eta 0:00:0000:0100:01


In [15]:
# data = datasets.load_dataset('sehyun66/STOCKPRICE','NASDAQ_3y')
import polars as pl


In [23]:
df =data['train'].to_pandas()


In [24]:
df

,Date,Open,High,Low,Close,Adj Close,Volume,Ticker
0,2020-11-23,538.859985,538.859985,516.799988,523.609985,523.609985,1764400,REGN
1,2020-11-24,521.609985,528.440002,505.799988,506.100006,506.100006,1270300,REGN
2,2020-11-25,507.179993,515.469971,507.179993,508.309998,508.309998,711800,REGN
3,2020-11-27,513.419983,518.919983,509.200012,514.049988,514.049988,498200,REGN
4,2020-11-30,520.000000,520.780029,507.600006,516.030029,516.030029,1707500,REGN
...,...,...,...,...,...,...,...,...
1612675,2023-11-16,190.229996,193.830002,187.020004,189.169998,189.169998,184000,BGNE
1612676,2023-11-17,189.169998,189.884995,184.880005,186.880005,186.880005,306000,BGNE
1612677,2023-11-20,186.889999,188.649994,184.419998,187.880005,187.880005,418100,BGNE
1612678,2023-11-21,185.300003,187.000000,183.675003,184.929993,184.929993,128800,BGNE


In [1]:


SUFFLE = False
WINDOW_SIZE  = 90
PREDICT_SIZE = 15
SLIDING_SIZE =1
FEATURE_SIZE = 5
BATCH_SIZE =64
FULLLY_SIZE =128
FEATURE = [ 'Open', 'High','Low','Close', 'Volume']
TARGET = 'Close'
TICKER_NUMBER =2136
TOTAL_DAY = 755
WINDOW_NUMBER =int(TOTAL_DAY - (WINDOW_SIZE+PREDICT_SIZE)/SLIDING_SIZE+1)



data= datasets.load_dataset('sehyun66/STOCKPRICE','NASDAQ_3y')
data = data['train'].to_pandas()


def min_max(sequences):
  results = sequences.copy()
  v_min =results.min()
  v_max =results.max()
  new_min =0
  new_max =1
  min_max=[]
  for index,sequence in enumerate(results):
    v_min =sequence.min()
    v_max =sequence.max()
    v_p = (sequence - v_min)/(v_max - v_min)*(new_max - new_min) + new_min
    min_max.append([v_min,v_max])
    results[index] = v_p
  return results, min_max

def inverse_min_max(sequences,min_max):
  original = []
  for index,sequence in enumerate(sequences):
    v_min =min_max[index][0]
    v_max =min_max[index][1]
    sequence_restored = (sequence) * (v_max - v_min) + v_min
    original.append(sequence_restored)
  return np.array(original)

def preprocess_lstm(df):
  sequences = list()
  for group in df.groupby('Ticker'):
    sequences.append(group[1][FEATURE])
  sequences=np.array(sequences)

  ## min_max trading
  price , trade,sentiments =np.split(sequences,[4,5],axis=2)
  trade , min_max_trading =min_max(trade)
  price , min_max_list =min_max(price)
  combine =np.concatenate([price,trade,sentiments],axis =2)

  ## min_max another
  result_list= []
  for i in range(0,WINDOW_NUMBER):
    a, b, c= np.split(combine,[i,i+WINDOW_SIZE+PREDICT_SIZE],axis=1)
    result_list.append(b)
  result_array = np.array(result_list)
  print(166//(WINDOW_SIZE+PREDICT_SIZE))
  # train ,vaild = np.split(result_array,[TOTAL_DAY%(WINDOW_SIZE+PREDICT_SIZE)],axis=0)
  train ,vaild = np.split(result_array,[-1],axis=0)

  print(train.shape ,vaild.shape)
  # train_x,train_y=np.split(train.reshape(TOTAL_DAY%(WINDOW_SIZE+PREDICT_SIZE)*TICKER_NUMBER,WINDOW_SIZE+PREDICT_SIZE,FEATURE_SIZE),[WINDOW_SIZE],axis=1)
  train_x,train_y=np.split(train.reshape((WINDOW_NUMBER-1)*TICKER_NUMBER,WINDOW_SIZE+PREDICT_SIZE,FEATURE_SIZE),[WINDOW_SIZE],axis=1)
  vaild_x,vaild_y=np.split(vaild.reshape(1*TICKER_NUMBER,WINDOW_SIZE+PREDICT_SIZE,FEATURE_SIZE),[WINDOW_SIZE],axis=1)
  return train_x,train_y,vaild_x,vaild_y,min_max_list



class BaseDataset(Dataset):
    def __init__(self, x_data, y_data, target):
        self.x_data = torch.tensor(x_data,dtype =torch.float32)
        self.y_data = torch.tensor(y_data[:,FEATURE.index(target),:],dtype = torch.float32)

    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return self.x_data.shape[0]
train_x,train_y,vaild_x,vaild_y,min_max_list = preprocess_lstm(data)

def swap_axes(arr):
    return np.swapaxes(arr, 1, 2)
train_x = swap_axes(train_x)
train_y = swap_axes(train_y)
vaild_x = swap_axes(vaild_x)
vaild_y = swap_axes(vaild_y)
train_data = BaseDataset(train_x,train_y,TARGET)
test_data =  BaseDataset(vaild_x,vaild_y,TARGET)
train_dataloader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=False,drop_last=True)
test_dataloader = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=False,drop_last=True)
train_x.shape,train_y.shape,vaild_x.shape,vaild_y.shape,len(min_max_list)





NameError: name 'datasets' is not defined

In [1]:
import datasets
import sys
sys.path.append("../../../")
from data.data_processing import preprocess_window



SUFFLE = True
WINDOW_SIZE  = 90
PREDICT_SIZE = 15
SLIDING_SIZE =1
FEATURE_SIZE = 5
BATCH_SIZE =64
FULLLY_SIZE =128
FEATURE = [ 'Open', 'High','Low','Close', 'Volume']
TARGET = 'Close'
TICKER_NUMBER =2136
TOTAL_DAY = 755
WINDOW_NUMBER =int(TOTAL_DAY - (WINDOW_SIZE+PREDICT_SIZE)/SLIDING_SIZE+1)
data= datasets.load_dataset('sehyun66/STOCKPRICE','NASDAQ_3y')
data = data['train'].to_pandas()
# def preprocess_window(df,feature:list,TARGET:str,window_size:int,predict_size:int,sliding_size:int):

train_x,train_y,vaild_x,vaild_y,min_max_list =preprocess_window(data,FEATURE,TARGET,WINDOW_SIZE,PREDICT_SIZE,SLIDING_SIZE)


/home/ssu36/anaconda3/envs/tiger/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


755
651
(650, 2136, 105, 5) (1, 2136, 105, 5)
651


In [22]:
class TCN_trainer:
    def __init__(self,model, train_loader, test_loader,num_epochs = None, lr = None,batch_size=BATCH_SIZE, verbose = 1, patience=None):
        self.model = model
        self.batch_size = batch_size
        self.train_loader = train_loader
        self.test_loader =test_loader
        self.verbose = verbose
        self.patience =patience
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.criterion = nn.MSELoss().to(self.device)
        self.optimizer =  optim.Adam(model.parameters(), lr = lr)
        self.scheduler =   optim.lr_scheduler.ReduceLROnPlateau(self.optimizer, 'min')
        self.nb_epochs = num_epochs
        self.train_hist  = np.zeros(self.nb_epochs)
        self.vaild_hist =[]

    def train(self):
        config = {
        "model":'TCN',
        "learning_rate": IR,
        "epochs": EPOCH,
        "batch_size": BATCH_SIZE,
        "shuffle":SUFFLE,
        'verbose':1,
        'patience':PATIENCE,
        'dropout':DROPOUT,
        'target':TARGET,
        'feature_columns':FEATURE,
        "NUM_CHANNELS":num_channels,
        'KERNEL':kenel_size,

        }
        run = wandb.init(project ="LSTM",config=config)

        self.model.to(self.device)
        for epoch in range(self.nb_epochs):
            avg_cost = 0
            total_batch = len(self.train_loader)
            for batch_idx, samples in enumerate(self.train_loader):
                x_train, y_train = samples
                x_train =x_train.cuda()
                y_train =y_train.cuda()
                outputs = self.model(x_train)
                loss = self.criterion(outputs, y_train)
                self.optimizer.zero_grad()
                ##loss
                loss.backward()
                self.optimizer.step()
                avg_cost += loss/total_batch
                ##정확도 계산
                predict, label = outputs.clone().detach(),y_train.clone().detach()
                predict = inverse_min_max(predict.to('cpu').numpy(),min_max_list)
                label =  inverse_min_max(label.to('cpu').numpy(),min_max_list)
                score = self.MAE(predict,label)
                acc = self.accuracy(label,predict)
                wandb.log({"Training Loss": loss.item()})
            self.train_hist[epoch] = avg_cost
            if epoch % self.verbose == 0:
                total_loss, score,acc =self.vaild()
                print('Epoch:', '%04d' % (epoch), 'train loss :', '{:.4f}'.format(avg_cost))
                print('vaild_loss:', '{:.4f}'.format (total_loss), 'MAE :', '{:.4f}'.format(score),'Accuarcy :', '{:.4f}'.format(acc))
                wandb.log({"Evaluation Loss": loss.item()})
                wandb.log({"Evaluation MAE": score.item()})
                wandb.log({"Evaluation Accuracy": acc.item()})
                self.vaild_hist.append({"vaild_loss":total_loss ,"vaild_score":score,"vaild_accuracy":acc,"epoch":epoch})
                self.scheduler.step(total_loss)
        # patience번째 마다 early stopping 여부 확인
            if (epoch % self.patience == 0) & (epoch != 0):
                # loss가 커졌다면 early stop
                index =int(epoch/self.patience)
                if index> 1:
                    print((self.vaild_hist[index-1]['vaild_loss'] ,  self.vaild_hist[index]['vaild_loss'])   )
                    if self.vaild_hist[index-1]['vaild_loss']<self.vaild_hist[index]['vaild_loss']:
                        print('\n Early Stopping')
                        break
                    else:
                        print('model was saved')
                        torch.save(self.model,"best-model.pt")
    def vaild(self):
        self.model.to(self.device)
        self.model.eval()
        with torch.no_grad():
            pred = []
            avg_cost = 0
            score = 0
            acc = 0
            total_batch= len(test_dataloader)
            for batch_idx, samples in enumerate(self.test_loader):
                x_test, y_test = samples
                x_test =x_test.cuda()
                y_test =y_test.cuda()
                outputs = self.model(x_test)
                loss = self.criterion(outputs, y_test)
                avg_cost += loss
                predict = inverse_min_max(outputs.to('cpu'),min_max_list)
                y_test =  inverse_min_max(y_test.to('cpu'),min_max_list)
                score += self.MAE(predict,y_test)
                acc += self.accuracy(y_test,predict)
        total_loss = avg_cost/total_batch
        score = score/len(self.test_loader)
        acc = acc/len(self.test_loader)
        self.model.train()
        return total_loss,score,acc

    def MAE(self,true, pred):
         return np.mean(np.abs(true-pred))
    def accuracy(self,true, pred):
        return (1-np.mean(np.abs((true-pred)/true)))




In [23]:
model = TCNModel(input_size=5 ,num_channels=num_channels,output_size=PREDICT_SIZE,kernel_size=kenel_size, dropout=0.25)
trainer = TCN_trainer(model,train_dataloader,test_dataloader,num_epochs=EPOCH,lr= IR,batch_size=BATCH_SIZE,patience=PATIENCE)


/home/ssu36/anaconda3/envs/tiger/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")
/home/ssu36/anaconda3/envs/tiger/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [24]:
trainer.train()
wandb.finish()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


wandb: Currently logged in as: sdfg8931 (sehyunai). Use `wandb login --relogin` to force relogin


Epoch: 0000 train loss : 1912651.5000
vaild_loss: 0.0081 MAE : 19.8371 Accuarcy : 0.7875
Epoch: 0001 train loss : 0.0045
vaild_loss: 0.0048 MAE : 11.0538 Accuarcy : 0.7425
Epoch: 0002 train loss : 0.0047
vaild_loss: 0.0049 MAE : 12.2545 Accuarcy : 0.7621


In [4]:
wandb.finish()

In [9]:
from dataclasses import dataclass

class TCNargumenets():
        input_size : int = 5
        output_size : int = 20
        num_channels : list = [80]*4
        kernel_size : int =7
        dropout : float = 0.25

In [32]:
params={
  'input_size' : 5,
  'output_size' : 20,
  'num_channels' : [80]*4,
  'kernel_size' : 7,
  'dropout' : 0.25
}

In [45]:
trgs = TCNargumenets()


from transformers import Trainer, TrainingArguments,HfArgumentParser


In [42]:
args=TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
)

In [2]:

@dataclass
class TCNargumenets(HfArgumentParser):
        input_size : int = 5
        output_size : int = 20
        num_channels : list = [80]*4
        kernel_size : int =7
        dropout : float = 0.25
parser = HfArgumentParser(TCNargumenets)

NameError: name 'dataclass' is not defined

In [1]:
parse = parser.parse_args()

NameError: name 'parser' is not defined

In [34]:
tcn_args = TCNargumenets()

model = TCNModel(**params)

/home/ssu36/anaconda3/envs/tiger/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


In [35]:
model

TCNModel(
  (tcn): TemporalConvNet(
    (network): Sequential(
      (0): TemporalBlock(
        (conv1): Conv1d(5, 80, kernel_size=(7,), stride=(1,), padding=(6,))
        (chomp1): Chomp1d()
        (relu1): ReLU()
        (dropout1): Dropout(p=0.25, inplace=False)
        (conv2): Conv1d(80, 80, kernel_size=(7,), stride=(1,), padding=(6,))
        (chomp2): Chomp1d()
        (relu2): ReLU()
        (dropout2): Dropout(p=0.25, inplace=False)
        (net): Sequential(
          (0): Conv1d(5, 80, kernel_size=(7,), stride=(1,), padding=(6,))
          (1): Chomp1d()
          (2): ReLU()
          (3): Dropout(p=0.25, inplace=False)
          (4): Conv1d(80, 80, kernel_size=(7,), stride=(1,), padding=(6,))
          (5): Chomp1d()
          (6): ReLU()
          (7): Dropout(p=0.25, inplace=False)
        )
        (downsample): Conv1d(5, 80, kernel_size=(1,), stride=(1,))
        (relu): ReLU()
      )
      (1): TemporalBlock(
        (conv1): Conv1d(80, 80, kernel_size=(7,), stride

In [11]:
tcn_args = TCNargumenets()

In [24]:
import argparse

# 인자값을 받을 수 있는 인스턴스 생성
parser = argparse.ArgumentParser(description='사용법 테스트입니다.')

# 입력받을 인자값 등록
parser.add_argument('--target', required=True, help='어느 것을 요구하냐')
parser.add_argument('--env', required=False, default='dev', help='실행환경은 뭐냐')

# 입력받은 인자값을 args에 저장 (type: namespace)
args = parser.parse_args()

usage: ipykernel_launcher.py [-h] --target TARGET [--env ENV]
ipykernel_launcher.py: error: the following arguments are required: --target


SystemExit: 2

/home/ssu36/anaconda3/envs/tiger/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3534: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [23]:
TCNModel(tcn_args.input_size, tcn_args.output_size, tcn_args.num_channels, tcn_args.kernel_size, tcn_args.dropout)

/home/ssu36/anaconda3/envs/tiger/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


TCNModel(
  (tcn): TemporalConvNet(
    (network): Sequential(
      (0): TemporalBlock(
        (conv1): Conv1d(5, 80, kernel_size=(7,), stride=(1,), padding=(6,))
        (chomp1): Chomp1d()
        (relu1): ReLU()
        (dropout1): Dropout(p=0.25, inplace=False)
        (conv2): Conv1d(80, 80, kernel_size=(7,), stride=(1,), padding=(6,))
        (chomp2): Chomp1d()
        (relu2): ReLU()
        (dropout2): Dropout(p=0.25, inplace=False)
        (net): Sequential(
          (0): Conv1d(5, 80, kernel_size=(7,), stride=(1,), padding=(6,))
          (1): Chomp1d()
          (2): ReLU()
          (3): Dropout(p=0.25, inplace=False)
          (4): Conv1d(80, 80, kernel_size=(7,), stride=(1,), padding=(6,))
          (5): Chomp1d()
          (6): ReLU()
          (7): Dropout(p=0.25, inplace=False)
        )
        (downsample): Conv1d(5, 80, kernel_size=(1,), stride=(1,))
        (relu): ReLU()
      )
      (1): TemporalBlock(
        (conv1): Conv1d(80, 80, kernel_size=(7,), stride